## __Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto__

#### Import the required packages and libraries

In [190]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

#### Scrape the wikipedia page and extract the Toronto table

In [191]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table', attrs= {'class': "wikitable sortable"})
table_row = table.find_all('tr')

res = []
for tr in table_row:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])

#### Find the index of row which Borough = Not assigned and drop those rows

In [192]:
ind = df[df['Borough'] == 'Not assigned'].index
df.drop(ind, axis = 0, inplace = True)

#### Combine rows containing same PostalCode with the neighborhoods separated with a comma

In [193]:
df_clean = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

#### Clean the column 'Neighborhood' for the case that a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [194]:
x = np.linspace(0,len(df_clean['Neighborhood'])-1, len(df_clean['Neighborhood'])).astype(int)
for n in x:
    if df_clean['Neighborhood'][n] == 'Not assigned':
        df_clean['Neighborhood'][n] = df_clean['Borough'][n]
    else:
        df_clean['Neighborhood'][n] = df_clean['Neighborhood'][n]

#### Print the number of rows in the dataframe

In [195]:
print('Total number of rows in the dataframe is', df_clean.shape[0])

Total number of rows in the dataframe is 103
